In [ ]:
from PIL import Image
import numpy as np
import cv2
import time
import sys

%run "common_functions.py"

sys.setrecursionlimit(53000) # override needed for computing midpoints, which uses a recursive function
Image.MAX_IMAGE_PIXELS = 366498276 # override is needed, or else it gives a DecompressionBombError

cy1_shifted_file = "originals/shiftedcycle 1thnksgv.tif"
cy2_shifted_file = "originals/shiftedcycle 2thnksgv.tif"

In [ ]:
def c(img):
    return Image.fromarray(img)

In [ ]:
layer_to_align = 2

img_cy1 = Image.open(cy1_shifted_file)
brightfield_cy1 = np.array(img_cy1)
brightfield_cy1 = (brightfield_cy1/256).astype('uint8') # don't need the whole range of values, so this reduces mem size, + improves access speed

img_cy2 = Image.open(cy2_shifted_file)
brightfield_cy2 = np.array(img_cy2)
brightfield_cy2 = (brightfield_cy2/256).astype('uint8') 

img_cy1 = None
img_cy2 = None

In [ ]:
# y,height = 0, 2500
# x, width = 0, 2500
# img1 = brightfield_cy1[y:height, x:width]
# img2 = brightfield_cy2[y:height, x:width]

img1 = brightfield_cy1
img2 = brightfield_cy2

# brightfield_cy1 = None
# brightfield_cy2 = None

In [ ]:
# def cutimage(img):
    

In [ ]:
def alignImages(im1, im2):
  # Convert images to grayscale
  #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
 
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(10000)
  keypoints1, descriptors1 = orb.detectAndCompute(im1, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2, None)
 
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = list(matcher.match(descriptors1, descriptors2, None))
 
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  print("Count of Matches:", len(matches))
  numGoodMatches = int(len(matches) * .05)
  print("Count of Good Matches:", numGoodMatches)
  print("Distance of worst match:", matches[numGoodMatches].distance)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  #imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  #cv2.imwrite("matches.jpg", imMatches)
 
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
 
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
 
  return im1Reg, h
 
num_alignments = 1
alignable = adjust_contrast(img1.copy())
print(alignable.shape)

# crop along boundaries
alignable[img2 == 0] = 0
img2[alignable == 0] = 0

for align_n in range(num_alignments):
  alignable, h = alignImages(alignable,adjust_contrast(img2))

c(adjust_contrast(img2))

In [ ]:

def gitkp(im1, im2):
  # Convert images to grayscale
  #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
 
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(100000)
  keypoints1, descriptors1 = orb.detectAndCompute(im1, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2, None)
 
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = list(matcher.match(descriptors1, descriptors2, None))
 
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  print("Count of Matches:", len(matches))
  numGoodMatches = int(len(matches) * .5)
  print("Count of Good Matches:", numGoodMatches)
  print("Distance of worst match:", matches[numGoodMatches].distance)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  #imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  #cv2.imwrite("matches.jpg", imMatches)
 
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
 
  imageresult1 = cv2.drawKeypoints(im1, keypoints1, None, color=(255,0,0), flags=0)
  imageresult2 = cv2.drawKeypoints(im2, keypoints2, None, color=(255,0,0), flags=0)
  return imageresult1, imageresult2
 
# imageresult1, imageresult2 = gitkp(adjust_contrast(img1),adjust_contrast(img2))
# 

In [ ]:
# c(imageresult1).save("kp1.png")
c(imageresult2).save("kp2.png")

In [ ]:
# c(imageresult1)

In [ ]:
# c(alignable)

In [ ]:
# c(alignable).save("cunty.png")

https://towardsdatascience.com/make-knn-300-times-faster-than-scikit-learns-in-20-lines-5e29d74e76bb
https://medium.com/aiguys/other-versions-of-knn-fast-nearest-neighbors-43a051bfb9aa
https://stackoverflow.com/questions/51688568/faster-knn-classification-algorithm-in-python#:~:text=KNN%20is%20a%20very%20slow,all%20the%20distances%20and%20sorting.


In [ ]:
4/0

In [ ]:
def adjust_contrast(img, min=2, max = 98):
    # pixvals = np.array(img)

    minval = np.percentile(img, min) # room for experimentation 
    maxval = np.percentile(img, max) # room for experimentation 
    img = np.clip(img, minval, maxval)
    img = ((img - minval) / (maxval - minval)) * 255
    return (img.astype(np.uint8))

def threshold(im, percentile):
    p = np.percentile(im, percentile)
    im[im < p]  = 0
    im[im >= p]  = 255
    
    return im



In [ ]:
cases = {
    0: "only neither is  solid", 
    1: "only cycle 1 is solid",
    2: "only cycle 2 is solid", 
    3: "both are solid",     
    }

In [ ]:
c(alignable)

In [ ]:
c(threshold(adjust_contrast(alignable), 90))

In [ ]:

def color_in(mat_orig, min_size=0):
    mat = (mat_orig)
    gray_edit_copy = mat_orig.copy()

    def fill_blob(x, y):
        
        mat[y][x] = False
        gray_edit_copy[y][x] = False
        
        #  right
        if  (x != mat.shape[1] - 1) and mat[y][x+1]:
            (fill_blob(x+1, y))
        # down
        if  (y != mat.shape[0] - 1) and mat[y+1][x]:
            (fill_blob(x, y+1))
        # left
        if  (x != 0) and mat[y][x-1]:
            (fill_blob(x-1, y))
        # up
        if  (y != 0) and  mat[y-1][x]:
            fill_blob(x, y-1)
        
    for y, line in enumerate(mat):
        for x, pixel in enumerate(line):
            if pixel == 3:
                fill_blob(x, y)
                
    
    return gray_edit_copy


In [ ]:


im1_beads = threshold(adjust_contrast(alignable), 90)
im2_beads = (threshold(adjust_contrast(img2), 90))

c(im1_beads)

# im1_beads[im1
# _beads == 255] = 1
# im2_beads[im2_beads == 255] = 2

# overlaid = np.add(im1_beads, im2_beads)

# c((overlaid)*70)

In [ ]:

im1_beads = threshold(adjust_contrast(alignable), 85)
im2_beads = (threshold(adjust_contrast(img2), 85))

im1_beads[im1_beads == 255] = 1
im2_beads[im2_beads == 255] = 2

overlaid = np.add(im1_beads, im2_beads)


# c(adjust_contrast(color_in(overlaid), 0, 100))
# c(im2_beads)


In [ ]:
overlaid_disjoint = color_in(overlaid)
c(overlaid_disjoint * 100).save('overlaid_disjoint.png')

In [ ]:
4 / 0

In [ ]:
def remove_blob(mat_orig, xy):
    mat = (mat_orig != 0)
    queue = mat_orig.copy()
    def fill_blob(x, y):
        queue[y][x] = 0
        mat[y][x] = False
        blob_elems = [[x, y]]
        #  right
        if  (x != mat.shape[1] - 1) and mat[y][x+1]:
            blob_elems += (fill_blob(x+1, y))
        # down
        if  (y != mat.shape[0] - 1) and mat[y+1][x]:
            blob_elems +=(fill_blob(x, y+1))
        # left
        if  (x != 0) and mat[y][x-1]:
            blob_elems +=(fill_blob(x-1, y))
        # up
        if  (y != 0) and  mat[y-1][x]:
            blob_elems +=(fill_blob(x, y-1))
        
        
        return blob_elems
    
    fill_blob(xy[0], xy[1])
    
    return queue

    

In [ ]:
import diplib as dip

# out = dip.AreaOpening(imm_cy1, filterSize=3, connectivity=1)
# c(np.array(out))

%run "common_functions.py"

midpoints_cy1, sizes_cy1, gray_edit_copy_cy1, filter1 =  get_center_points(imm_cy1, 5, filter=True, gray=False)
midpoints_cy2, sizes_cy2, gray_edit_copy_cy2, filter2 =  get_center_points(imm_cy2, 5, filter=True, gray=False)

filter1[filter1 != 255] = 0
filter2[filter2 != 255] = 0

filter1[filter1 == 255] = 1
filter2[filter2 == 255] = 2

overlaid_filtered = np.add(filter1, filter2)

# filter1[filter1 == 1] = 255           
# filter2[filter2 == 2] = 255

print(len(midpoints_cy1))
print(len(midpoints_cy2))

# c(filter2*100)
c(np.add(filter1, filter2) * 100).save('cc.png')

In [ ]:
from scipy.spatial.distance import cdist

def closest_node(node, nodes):
    return nodes[cdist([node], nodes).argmin()]
# print(midpoints_cy2)

lines_between = (overlaid_filtered.copy()) * 100



if len(lines_between.shape) == 2:
        lines_between = np.stack((lines_between,)*3, axis=2)

cleaned_im1 = filter1.copy()
cleaned_im2 = filter2.copy()

import math
DISTANCE_THRESHOLD = 10

for i, mp in  enumerate(midpoints_cy2):
    cn = closest_node(mp, midpoints_cy1)
    # print(i)
    # print(math.dist(mp, cn))
    if math.dist(mp, cn) < DISTANCE_THRESHOLD:
        cleaned_im2 = remove_blob(cleaned_im2, mp)
        cleaned_im1 = remove_blob(cleaned_im1, cn)
        
    #     lines_between = cv2.line(lines_between, mp, cn, (0, 255, 0), 1) 
    # else:
    #     lines_between = cv2.line(lines_between, mp, cn, (255, 0, 0), 1) 

    # if i == 200:
    #     break
    


In [ ]:
c(lines_between)

In [ ]:
c(cleaned_im1*80)

In [ ]:
# https://docs.opencv.org/3.4/d2/d2c/tutorial_sobel_derivatives.html
grad_x = cv.Sobel(adjust_contrast(im1_r_onto_im2_third4), cv.CV_16S, 1, 0, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)
    # Gradient-Y
    # grad_y = cv.Scharr(gray,ddepth,0,1)
grad_y = cv.Sobel(adjust_contrast(im1_r_onto_im2_third4), cv.CV_16S, 0, 1, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)

abs_grad_x = cv.convertScaleAbs(grad_x)
abs_grad_y = cv.convertScaleAbs(grad_y)
    
    ``
grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    
num = 200
# c(adjust_contrast(grad))
grad[grad >= num] = 255
grad[grad < num] = 0
c(grad)
# c(threshold(adjust_contrast(adjust_contrast(im1_r_onto_im2_third4)), 80))

In [ ]:
ddepth = cv.CV_16S
kernel_size = 3
window_name = "Laplace Demo"

src = im1_r_onto_im2_third4.copy()
src = adjust_contrast(src)


dst = cv.Laplacian(src, ddepth, ksize=kernel_size)
# [laplacian]
# [convert]
# converting back to uint8
abs_dst = cv.convertScaleAbs(dst)


def blur_noise_reduction(im):


    im1 = im1.filter(ImageFilter.BLUR)
    # im1 = im1.filter(ImageFilter.BLUR)
    # im1 = im1.filter(ImageFilter.BLUR)
    # im1 = im1.filter(ImageFilter.BLUR)
    # im1 = im1.filter(ImageFilter.BLUR)
    # im1 = im1.filter(ImageFilter.BLUR)
    # im1 = im1.filter(ImageFilter.BLUR).filter(ImageFilter.BLUR).filter(ImageFilter.BLUR).filter(ImageFilter.BLUR).filter(ImageFilter.BLUR).filter(ImageFilter.BLUR)
    # im1 = im1 - np.array(diplib.AreaOpening(np.array(im1), filterSize=25, connectivity=2))
    


    return im1

from PIL import ImageFilter
# c(threshold(adjust_contrast(np.array(c(abs_dst).filter(ImageFilter.BLUR))), 80))
c((abs_dst))


In [ ]:


flow = cv.calcOpticalFlowFarneback(np.array(im1_r_onto_im2_third4), 
                                   np.array(img2), 
                                   None, 0.5, 3, 15, 3, 5, 1.2, 0)

def put_optical_flow_arrows_on_image(image, optical_flow_image, threshold=2.0, skip_amount=30):
    # Don't affect original image
    image = image.copy()
    
    # Turn grayscale to rgb if needed
    if len(image.shape) == 2:
        image = np.stack((image,)*3, axis=2)
    
    # Get start and end coordinates of the optical flow
    flow_start = np.stack(np.meshgrid(range(optical_flow_image.shape[1]), range(optical_flow_image.shape[0])), 2)
    flow_end = (optical_flow_image[flow_start[:,:,1],flow_start[:,:,0],:1]*3 + flow_start).astype(np.int32)
    

    # Threshold values
    norm = np.linalg.norm(flow_end - flow_start, axis=2)
    norm[norm < threshold] = 0
    
    # Draw all the nonzero values
    nz = np.nonzero(norm)
    for i in range(0, len(nz[0]), skip_amount):
        y, x = nz[0][i], nz[1][i]
        cv2.arrowedLine(image,
                        pt1=tuple(flow_start[y,x]), 
                        pt2=tuple(flow_end[y,x]),
                        color=(0, 255, 255), 
                        thickness=1, 
                        tipLength=.2)
    return image

c(put_optical_flow_arrows_on_image(im1_r_onto_im2_third4,flow))